In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
from pprint import pprint
from pathlib import Path
from random import randint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
from tqdm.notebook import tqdm
import nibabel as nib
import glmsingle
from glmsingle.glmsingle import GLM_single
import bids
from bids import BIDSLayout
from scipy.ndimage import zoom, binary_dilation
import h5py
import nibabel as nib
from einops import rearrange

dir2 = os.path.abspath('..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)
    
from noise_ceiling import (
    compute_ncsnr,
    compute_nc,
    group_repetitions
)
from kamitani import load_data, convert_ids

In [ ]:
# Enter the path to the Kamitani dataset
derivatives_path = Path('X:\\Datasets\\Deep-Image-Reconstruction\\derivatives\\')
derivatives_path_ssd = Path('D:\\Datasets\\Deep-Image-Reconstruction\\derivatives\\')
dataset_path = derivatives_path / 'fmriprep-20.2.4'

In [ ]:
# Load layouts, this will take about a minute because pybids is slow
t = time.time()
dataset_layout = bids.BIDSLayout(dataset_path)
print(time.time() - t)

In [ ]:
# Crop and prepare the data
subjects = ['02', '03']
#sessions = [f'perceptionNaturalImageTraining{i:02}' for i in range(1, 16)]
sessions = [f'perceptionNaturalImageTest{i:02}' for i in range(1, 4)]
space = 'T1w'

tr = 2.
num_runs = len(sessions) * 8
num_trs = 239
num_stimuli = 50

trend_coeffs = np.stack([np.arange(num_trs), np.ones(shape=num_trs)], axis=1)

mask_dilations = 3

with h5py.File(derivatives_path_ssd / 'kamitani-test-bold.hdf5', 'w') as f:
    for subject in tqdm(subjects):
        group = f.require_group(f'sub-{subject}')
        mask_image = dataset_layout.get(
            subject=subject, 
            session='perceptionNaturalImageTraining01', 
            space=space, 
            run=1, 
            desc='brain', 
            extension='nii.gz'
        )[0].get_image()
        fmri_mask = mask_image.get_fdata().astype(bool)
        fmri_mask = binary_dilation(fmri_mask, iterations=mask_dilations)
        num_voxels = fmri_mask.sum()
        
        if 'affine' not in group:
            group['affine'] = mask_image.affine
        
        H, W, D = fmri_mask.shape
        if 'fmri_mask' not in group:
            group['fmri_mask'] = fmri_mask
        
        group.require_dataset('bold', shape=(num_runs, num_trs, num_voxels), dtype='f4')
        group.require_dataset('bold_mean', shape=(num_runs, num_voxels), dtype='f4')
        group.require_dataset('bold_std', shape=(num_runs, num_voxels), dtype='f4')
        group.require_dataset('bold_trend', shape=(num_runs, 2, num_voxels), dtype='f4')
        group.require_dataset('bold_trend_std', shape=(num_runs, num_voxels), dtype='f4')
        group.require_dataset('stimulus_trs', shape=(num_runs, num_stimuli), dtype='i4')
        group.require_dataset('stimulus_ids', shape=(num_runs, num_stimuli), dtype='f8')
        group.require_dataset('num_runs_per_session', shape=(len(sessions),), dtype='i4')
        
        fmri_data = []
        stimulus_ids = []
        run_id = 0
        for session_id, session in tqdm(enumerate(sessions)):
            events_files = dataset_layout.get(
                subject=subject, 
                session=session, 
                extension='tsv', 
                suffix='events',
            )
            image_files = dataset_layout.get(
                subject=subject, 
                session=session, 
                space=space, 
                desc='preproc',
                suffix='bold',
                extension='nii.gz'
            )
            group['num_runs_per_session'][session_id] = len(image_files)

            for events_file, bids_image in tqdm(list(zip(events_files, image_files))):
                bold = bids_image.get_image().get_fdata()
                bold = bold[fmri_mask].T
                
                bold_trend = np.linalg.lstsq(trend_coeffs, bold, rcond=None)[0]
                bold_predicted = trend_coeffs @ bold_trend
                bold_detrend = bold - bold_predicted

                events_df = pd.read_csv(events_file.path, sep='\t', dtype={'stimulus_id': str})
                events_df = events_df[events_df['event_type'] == 1]
                stimulus_trs = np.rint(np.array(events_df['onset'] / tr)).astype(int)
                stimulus_ids = np.array(events_df['stimulus_id']).astype(float)
                
                group['bold'][run_id] = bold
                group['bold_mean'][run_id] = bold.mean(axis=0)
                group['bold_std'][run_id] = bold.std(axis=0)
                group['bold_trend'][run_id] = bold_trend
                group['bold_trend_std'][run_id] = bold_detrend.std(axis=0)
                group['stimulus_trs'][run_id] = stimulus_trs
                group['stimulus_ids'][run_id] = stimulus_ids
                
                run_id += 1
        

In [ ]:
derivatives_path = Path('D:\\Datasets\\Deep-Image-Reconstruction\\derivatives\\')
bold, stimulus_ids, mask, affine = load_data(
    derivatives_path / 'kamitani-bold.hdf5', 
    'sub-02', 
    tr_offset=3,
    run_normalize='linear_trend',
    session_normalize=False
)
bold.shape

In [ ]:
bold_volume = np.zeros_like(mask, dtype=float)
D = nc_volume.shape[2]
@interact(b=(0, bold.shape[0]-1), d=(0, D-1))
def show(b, d):
    plt.figure(figsize=(12, 12))
    bold_volume[mask] = bold[b]
    plt.imshow(bold_volume[:, :, d], cmap='bwr', vmin=-2, vmax=2,)

In [ ]:
ncsnr = compute_ncsnr(bold, group_repetitions(stimulus_ids, num_repetitions=5))
nc = compute_nc(ncsnr, num_averages=1)
nc_volume = np.zeros_like(mask, dtype=float)
nc_volume[mask] = nc

In [ ]:
D = nc_volume.shape[2]
@interact(d=(0, D-1), original=True)
def show(d):
    plt.figure(figsize=(12, 12))
    plt.imshow(nc_volume[:, :, d], cmap='jet', vmin=0., vmax=30,)

In [ ]:
# Compare normalizing runs by zscoring versus linear trends

subjects = ['sub-02', 'sub-03']
run_normalize_modes = ['zscore', 'linear_trend']
tr_window = (0, 8)

#results = {}
for subject in subjects:
    subject_path = derivatives_path_ssd / 'noise_ceiling' / subject
    subject_path.mkdir(exist_ok=True, parents=True)
    for run_normalize in run_normalize_modes:
        window_str = f'{tr_window[0] * 2}-{tr_window[1] * 2}'
        out_file_name = f'{subject}__window-{window_str}__run_normalize-{run_normalize}__noise-ceiling.nii.gz'
        if out_file_name in results:
            continue
        
        nc_series = []
        for tr_offset in tqdm(range(tr_window[0], tr_window[1] + 1)):
            bold, stimulus_ids, mask, affine = load_data(
                derivatives_path_ssd / 'kamitani-bold.hdf5', 
                subject,
                tr_offset=tr_offset,
                run_normalize=run_normalize
            )
            ncsnr = compute_ncsnr(bold, repetition_shape(stimulus_ids, n=5))
            nc = compute_nc(ncsnr, num_averages=1)
            nc_volume = np.zeros_like(mask, dtype=float)
            nc_volume[mask] = nc
            nc_series.append(nc_volume)
        nc_series = np.stack(nc_series, axis=-1)
        
        results[out_file_name] = nc_series
        image = nib.Nifti1Image(nc_series, affine)
        nib.save(image, subject_path / out_file_name)
        

In [ ]:
subjects = ['sub-02', 'sub-03']
tr_window = (0, 8)

results = {}
for subject in subjects:
    subject_path = derivatives_path_ssd / 'noise_ceiling' / subject
    subject_path.mkdir(exist_ok=True, parents=True)
    window_str = f'{tr_window[0] * 2}-{tr_window[1] * 2}'
    out_file_name = f'{subject}__window-{window_str}__session_normalize__noise-ceiling.nii.gz'

    nc_series = []
    for tr_offset in tqdm(range(tr_window[0], tr_window[1] + 1)):
        bold, stimulus_ids, mask, affine = load_data(
            derivatives_path_ssd / 'kamitani-bold.hdf5', 
            subject,
            tr_offset=tr_offset,
            run_normalize='linear_trend',
            session_normalize=True
        )
        ncsnr = compute_ncsnr(bold, repetition_shape(stimulus_ids, n=5))
        nc = compute_nc(ncsnr, num_averages=1)
        nc_volume = np.zeros_like(mask, dtype=float)
        nc_volume[mask] = nc
        nc_series.append(nc_volume)
    nc_series = np.stack(nc_series, axis=-1)

    results[out_file_name] = nc_series
    image = nib.Nifti1Image(nc_series, affine)
    nib.save(image, subject_path / out_file_name)

In [ ]:
subjects = ['sub-02', 'sub-03']
tr_window = (0, 8)

results = {}
for subject in subjects:
    subject_path = derivatives_path_ssd / 'noise_ceiling' / subject
    subject_path.mkdir(exist_ok=True, parents=True)
    window_str = f'{tr_window[0] * 2}-{tr_window[1] * 2}'
    out_file_name = f'{subject}__window-{window_str}__test__noise-ceiling.nii.gz'

    nc_series = []
    for tr_offset in tqdm(range(tr_window[0], tr_window[1] + 1)):
        bold, stimulus_ids, mask, affine = load_data(
            derivatives_path_ssd / 'kamitani-test-bold.hdf5', 
            subject,
            tr_offset=tr_offset,
            run_normalize='linear_trend',
            session_normalize=False
        )
        ncsnr = compute_ncsnr(bold, group_repetitions(stimulus_ids, num_repetitions=24))
        nc = compute_nc(ncsnr, num_averages=1)
        nc_volume = np.zeros_like(mask, dtype=float)
        nc_volume[mask] = nc
        nc_series.append(nc_volume)
    nc_series = np.stack(nc_series, axis=-1)

    results[out_file_name] = nc_series
    image = nib.Nifti1Image(nc_series, affine)
    nib.save(image, subject_path / out_file_name)

In [ ]:
nc_series = np.stack(nc_series, axis=-1)
results[out_file_name] = nc_series
image = nib.Nifti1Image(nc_series, affine)
nib.save(image, subject_path / out_file_name)